### Een parser-generator voor de wordgrammar

In de ETCBC-data wordt een morphologische analyse-annotatie gebruikt, die per project kan worden gedefinieerd in een `word_grammar`-definitiebestand. Per project moet er eerst een annotatieparser worden gegenereerd aan de hand van het `word-grammar`-bestand. Dat gebeurt in de `WordGrammar` class in `wrdgrm.py`, die afhankelijk is van de parser-generator in de `wgr.py` en `yapps-runtime.py` modules. De parser-generator is gegenereerd met Yapps2 (zie de website: http://theory.stanford.edu/~amitp/yapps/ en https://github.com/smurfix/yapps).

Om een `WordGrammar`-object te maken zijn een `word_grammar`-bestand en een `lexicon`-bestand vereist. Vervolgens kunnen woorden geanalyseerd worden met de method `WordGrammar.analyze(word)`.

In [1]:
# eerst modules importeren
import os
from wrdgrm import WordGrammar

hulpfunctie

In [2]:
def filepath(rel_path):
    return os.path.realpath(os.path.join(os.getcwd(), rel_path))

In [3]:
# bestandslocaties
lexicon_file = filepath("../../data/blc/syrlex")
word_grammar_file = filepath("../../data/blc/syrwgr")
an_file = filepath("../../data/blc/Laws.an")

In [4]:
# dan kan de wordgrammar worden geïnitialiseerd
wg = WordGrammar(word_grammar_file, lexicon_file)

De method `analyze()` retourneert een `Word`-object met de analyse.

In [5]:
# wrdgrm.Word object
wg.analyze(">TR/&WT=~>")

In [6]:
# voorbeeld
word = wg.analyze(">TR/&WT=~>")
print(
    "{:15}".format("Morphemes:"),
    tuple((m.mt.ident, (m.p, m.s, m.a)) for m in word.morphemes),
)
print("{:15}".format("Functions:"), word.functions)
print("{:15}".format("Lexicon:"), word.lex)
print("{:15}".format("Lexeme:"), word.lexeme)
print("{:15}".format("Annotated word:"), word.word)
print("{:15}".format("Meta form:"), word.meta_form)
print("{:15}".format("Surface form:"), word.surface_form)
print("{:15}".format("Paradigmatic form:"), word.paradigmatic_form)

Morphemes:      (('lex', ('>TR', '>TR', '>TR')), ('nme', ('T=', 'WT', '&WT=')), ('emf', ('>', '>', '>')))
Functions:      (('vt', False), ('vs', False), ('ps', False), ('sp', 'subs'), ('nu', 'pl'), ('gn', 'm'), ('st', 'emph'))
Lexicon:        ('17789', (('sp', 'subs'), ('gn', 'm'), ('gl', 'place, region')))
Lexeme:         >TR
Annotated word: >TR/&WT=~>
Meta form:      >TR&WT=>
Surface form:   >TRWT>
Paradigmatic form: >TRT=>


Naast verschillende `string`-weergaven van het geanalyseerde woord bevat het `Word`-object drie `tuples`: `morphemes`, `functions` en `lex`, met daarin de belangrijkste analyses.

De eerste, `morphemes`, bevat een tuple met alle gevonden morfemen, elk als een ~~tuple met drie strings~~ `Morpheme` object met vier attributen: `mt`, een namedtuple met informatie over het morfeemtype; `p`, de paradigmatische vorm (zoals die in het lexicon staat); `s`, de oppervlaktevorm (zoals die in de tekst staat); en `a`, de geannoteerde vorm met meta-karakters.

De tweede, `functions`, bevat de grammaticale functies van het woord, zoals die in de `wordgrammar` gedefinieerd zijn: `ps: "person"`, `nu: "number"`, `gn: "gender"`, `ls: "lexical set"`, `sp: "part of speech"`, `st: "state"`, `vo: "voice"`, `vs: "verbal stem"`, `vt: "verbal tense"`. Een veld met de waarde `False` geeft aan dat deze functie niet van toepassing is op dit woord, een veld met waarde `None` geeft aan dat de waarde niet is vastgesteld.

De derde, `lex`, bevat het lemma zoals dat in het lexicon staat, met als eerste het woord-id, en vervolgens de annotaties. Behalve standaard-waarden voor de grammaticale functies bevat het lexicon een `gl`-veld voor ieder woord (gloss), en soms een `de`-veld (derived form). (In één resp. twee gevallen komen ook de velden `cs` en `ln` voor, waarvan de betekenis mij niet duidelijk is.)

In [7]:
# De method `dmp_str` genereert een string die overeenkomt met die in .dmp-bestanden.
# Hieronder een voorbeeld hoe die gebruikt kan worden om een .dmp-bestand te genereren.
# Voor een eenvoudiger manier, zie de AnParser notebook.


def dump_anfile(name, an_file):
    with open(an_file) as f:
        for line in f:
            verse, s, a = line.split()  # verse, surface form, analyzed form
            for an_word in a.split("-"):
                word = wg.analyze(an_word)
                yield word.dmp_str(name, verse)


for i, line in zip(range(20), dump_anfile("Laws", an_file)):
    # for line in dump_anfile('Laws', an_file):
    print(line)
print("...")

Laws 0,1	TWB	TWB	TWB	-	sp=advb
Laws 0,1	KTB=/~>	KTB>	KTB=	nme="",emf=">"	sp=subs,+nu,gn=m,st=emph
Laws 0,1	D	D	D	-	sp=prep,ls=pcon
Laws 0,1	NMWS/(J~>	NMWS>	NMWS	nme="J",emf=">"	sp=subs,nu=pl,gn=m,st=emph
Laws 0,1	D	D	D	-	sp=prep,ls=pcon
Laws 0,1	>TR/&WT=~>	>TRWT>	>TR	nme="T=",emf=">"	sp=subs,nu=pl,gn=m,st=emph
Laws 1,1	MN	MN	MN	-	sp=prep
Laws 1,1	QDM	QDM	QDM	-	sp=prep
Laws 1,1	JWM/T=~>	JWMT>	JWM	nme="T=",emf=">"	sp=subs,nu=pl,gn=m,st=emph
Laws 1,1	<L=[/JN	<LJN	<L=	vbe="",nme="JN"	sp=verb,nu=pl,gn=m,st=abs,vo=act,vs=pe,vt=ptc
Laws 1,1	HWJ[N	HWJN	HWJ	vbe="N"	nu=pl,sp=verb,vo=act,vs=pe,vt=pf,ps=first,ls=vbex
Laws 1,1	L	L	L	-	sp=prep
Laws 1,1	!M!S<R=[/	MS<R	S<R=	pfm="M",vbe="",nme=""	sp=verb,+st,vo=act,vs=pe,vt=inf
Laws 1,1	L	L	L	-	sp=prep
Laws 1,1	CMCGRM/	CMCGRM	CMCGRM	nme=""	sp=subs,+nu,+gn,st=abs,ls=prop
Laws 1,1	>X/&W	>XW	>X	nme=""	sp=subs,+nu,+gn,+st
Laws 1,1	N	N	N	-	nu=pl,ps=first,sp=pron,ls=pers
Laws 1,1	W	W	W	-	sp=conj
Laws 1,1	>T(J&>[	>T>	>TJ	vbe=""	nu=sg,gn=m,sp=verb,vo=act,vs=pe

Om te controleren dat de output correct is heb ik bovenstaande output vergeleken met de bestaande .dmp-bestanden. Omdat de volgorde van de waarden willekeurig lijkt te zijn - of in ieder geval niet in alle gevallen gelijk - moeten alle waarden gesorteerd worden voor ze vergeleken kunnen worden, een eenvoudige diff volstaat niet. Onderstaand script ~~bevestigt dat bovenstaande output, op de volgorde na, een exacte weergave is van de bestaande .dmp-bestanden~~ toont aan dat zowel de an-file als de word_grammar zijn aangepast sinds de .dmp-bestanden zijn gegenereerd:

(verschillen: woorden met vpm=dp zijn nu correct als vo=pas geanalyseerd, en van `]>](NKJ[` in 15,12 en `]M]SKN[/JN` in 19.12 zijn de annotaties gewijzigd)

In [8]:
dmp_file = filepath("../../data/blc/Laws.dmp")
dmp_gen = dump_anfile("BLC", an_file)

with open(dmp_file) as f_dmp:
    for line1, line2 in zip(f_dmp, dmp_gen):
        for f1, f2 in zip(line1.strip().split("\t"), line2.split("\t")):
            f1s, f2s = (",".join(sorted(f.split(","))) for f in (f1, f2))
            if f1s != f2s:
                print(f"{line1}!=\n{line2}")

BLC 1,4	!M!PQD[/JN:dp	MPQDJN	PQD	pfm="M",vbe="",nme="JN",vpm=dp	sp=verb,nu=pl,gn=m,st=abs,vo=act,vs=pa,vt=ptc
!=
BLC 1,4	!M!PQD[/JN:dp	MPQDJN	PQD	pfm="M",vbe="",nme="JN",vpm=dp	sp=verb,nu=pl,gn=m,st=abs,vo=pas,vs=pa,vt=ptc
BLC 8,20	!M!TQN[/:dp	MTQN	TQN	pfm="M",vbe="",nme="",vpm=dp	sp=verb,+nu,+gn,+st,vo=act,vs=pa,vt=ptc
!=
BLC 8,20	!M!TQN[/:dp	MTQN	TQN	pfm="M",vbe="",nme="",vpm=dp	sp=verb,+nu,+gn,+st,vo=pas,vs=pa,vt=ptc
BLC 10,2	!M!CLV[/JN:dp	MCLVJN	CLV	pfm="M",vbe="",nme="JN",vpm=dp	sp=verb,nu=pl,gn=m,st=abs,vo=act,vs=pa,vt=ptc
!=
BLC 10,2	!M!CLV[/JN:dp	MCLVJN	CLV	pfm="M",vbe="",nme="JN",vpm=dp	sp=verb,nu=pl,gn=m,st=abs,vo=pas,vs=pa,vt=ptc
BLC 15,12	]>](NKJ[/	>KJ	NKJ	vbs=">",vbe="",nme=""	sp=verb,+nu,+gn,+st,vo=act,vs=af
!=
BLC 15,12	]>](NKJ[	>KJ	NKJ	vbs=">",vbe=""	nu=sg,gn=m,sp=verb,vo=act,vs=af,vt=pf,ps=third
BLC 15,12	]>](NKJ[/	>KJ	NKJ	vbs=">",vbe="",nme=""	sp=verb,+nu,+gn,+st,vo=act,vs=af
!=
BLC 15,12	]>](NKJ[	>KJ	NKJ	vbs=">",vbe=""	nu=sg,gn=m,sp=verb,vo=act,vs=af,vt=pf,ps=third
B